# compute kinetic energies

including low frequency contributions

In [1]:
import os, sys, shutil
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

import xgcm

from mitequinox.utils import *
from mitequinox.dynamic import *
from mitequinox.plot import *

In [2]:
from dask_jobqueue import PBSCluster
cluster = PBSCluster(cores=1)
w = cluster.scale(30)
#w = cluster.scale(5) # postprocessing

In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.135.39.90:53366 Dashboard: http://10.135.39.90:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


---
# full fields


In [5]:
grd = load_grd().drop(['hFacC','hFacW','hFacS','rA','rAw','rAs'])

dsU = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('SSU'))
dsV = xr.open_zarr(work_data_dir+'rechunked/%s_std.zarr'%('SSV'))

#ds0 = xr.merge([dsU,dsV])

u2 = (dsU.SSU**2).mean('time').rename('SSU2')
v2 = (dsV.SSV**2).mean('time').rename('SSV2')
mu = dsU.SSU.mean('time').rename('mSSU')
mv = dsV.SSV.mean('time').rename('mSSV')

ds = xr.merge([u2,v2,mu,mv])

print(ds)

<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320)
Coordinates:
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
Data variables:
    SSU2     (face, j, i_g) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    SSV2     (face, j_g, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    mSSU     (face, j, i_g) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>
    mSSV     (face, j_g, i) float32 dask.array<shape=(13, 4320, 4320), chunksize=(1, 4320, 4320)>


In [9]:
ds.to_zarr(work_data_dir+'filtered/u2v2mumv.zarr', mode='w')

distributed.scheduler - ERROR - '9982695'
Traceback (most recent call last):
  File "/home/mp/pontea/.conda/envs/equinox/lib/python3.6/site-packages/distributed/scheduler.py", line 1306, in add_worker
    plugin.add_worker(scheduler=self, worker=address)
  File "/home/mp/pontea/.conda/envs/equinox/lib/python3.6/site-packages/dask_jobqueue/core.py", line 62, in add_worker
    self.running_jobs[job_id] = self.pending_jobs.pop(job_id)
KeyError: '9982695'


KilledWorker: ("('zarr-3bbdca6e867e7c516713529c62b86fa3', 484, 3, 0, 0)", 'tcp://10.135.36.34:39140')

In [6]:
for face in range(13):
    
    print('--- start processing face %d'%face)

    ldsU = dsU.sel(face=face)
    ldsV = dsV.sel(face=face)

    u2 = (ldsU.SSU**2).mean('time').rename('SSU2')
    v2 = (ldsV.SSV**2).mean('time').rename('SSV2')
    mu = ldsU.SSU.mean('time').rename('mSSU')
    mv = ldsV.SSV.mean('time').rename('mSSV')

    ds = xr.merge([u2,v2,mu,mv])

    ds.to_zarr(work_data_dir+'filtered/u2v2mumv_f%02d.zarr'%face, mode='w')    
    
    print('statistics done for face=%d'%(face))

--- start processing face 0
statistics done for face=0
--- start processing face 1
statistics done for face=1
--- start processing face 2


KilledWorker: ("('zarr-getitem-ffe132ca01942e9286bab79b79ca20cc', 3513, 0, 0)", 'tcp://10.135.36.45:34332')

---

## same but with subdiurnal fields

In [ ]:
for face in range(13):
    
    print('--- start processing face %d'%face)

    dsU = xr.open_zarr(work_data_dir+'filtered/%s_subd_%02d.zarr'%('SSU',face))
    dsV = xr.open_zarr(work_data_dir+'filtered/%s_subd_%02d.zarr'%('SSV',face))

    u2 = (dsU.SSU**2).mean('time').rename('SSU2')
    v2 = (dsV.SSV**2).mean('time').rename('SSV2')
    mu = dsU.SSU.mean('time').rename('mSSU')
    mv = dsV.SSV.mean('time').rename('mSSV')

    ds = xr.merge([u2,v2,mu,mv])

    ds.to_zarr(work_data_dir+'filtered/u2v2mumv_subd_f%02d.zarr'%face, mode='w')    
    
    print('statistics done for face=%d'%(face))

---

In [17]:
w = cluster.scale_up(30)

In [ ]:
client.restart()

In [3]:
cluster.close()